# Setup

In [1]:
from __future__ import print_function
from __future__ import division
from __future__ import unicode_literals

In [2]:
import pyspark.sql.functions as F
from pyspark.sql import Row

In [3]:
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.feature import Imputer, VectorAssembler, StandardScaler

In [4]:
from tykuo_spark_model import StringDisassembler, ImputeCategoricalWithMode, VectorDisassembler

# Sample Data

In [5]:
df = sc \
    .parallelize([
        (1, 'a', 'A', 56., 175.), 
        (2, 'a', 'B', 66., None), 
        (3, 'b', 'B', None, 182.), 
        (4, 'c', None, 71., 171.), 
        (5, None, 'B', 48., 173.)]) \
    .toDF(["id", "x1", 'x2', 'x3', 'x4'])

In [6]:
df.show()

+---+----+----+----+-----+
| id|  x1|  x2|  x3|   x4|
+---+----+----+----+-----+
|  1|   a|   A|56.0|175.0|
|  2|   a|   B|66.0| null|
|  3|   b|   B|null|182.0|
|  4|   c|null|71.0|171.0|
|  5|null|   B|48.0|173.0|
+---+----+----+----+-----+



# Data Processing Pipeline

In [7]:
stage = [
    # impute
    ImputeCategoricalWithMode().setInputCols(['x1', 'x2']).setOutputCols(['x1_im', 'x2_im']),
    Imputer(inputCols=['x3', 'x4'], outputCols=['x3_im', 'x4_im']).setStrategy('median'),
    # Numeric 
    VectorAssembler(inputCols=['x3_im', 'x4_im'], outputCol='numericFeatures'),
    StandardScaler(inputCol='numericFeatures', outputCol='scaledNumericFeatures', withMean=True),
    VectorDisassembler().setInputCol('scaledNumericFeatures').setOutputCols(['x3_scaled', 'x4_scaled']),
    # Categorical
    StringDisassembler().setInputCol('x1_im'),
    StringDisassembler().setInputCol('x2_im'),
]

In [8]:
pipelineModel = Pipeline(stages=stage).fit(df)

In [9]:
df_with_features = pipelineModel.transform(df)

In [10]:
df_with_features.printSchema()

root
 |-- id: long (nullable = true)
 |-- x1: string (nullable = true)
 |-- x2: string (nullable = true)
 |-- x3: double (nullable = true)
 |-- x4: double (nullable = true)
 |-- x1_im: string (nullable = true)
 |-- x2_im: string (nullable = true)
 |-- x3_im: double (nullable = true)
 |-- x4_im: double (nullable = true)
 |-- numericFeatures: vector (nullable = true)
 |-- scaledNumericFeatures: vector (nullable = true)
 |-- x3_scaled: double (nullable = true)
 |-- x4_scaled: double (nullable = true)
 |-- is_x1_im_a: double (nullable = true)
 |-- is_x1_im_c: double (nullable = true)
 |-- is_x1_im_b: double (nullable = true)
 |-- is_x2_im_A: double (nullable = true)
 |-- is_x2_im_B: double (nullable = true)



In [11]:
df_with_features_pd = df_with_features.toPandas().set_index('id')
df_with_features_pd

,x1,x2,x3,x4,x1_im,x2_im,x3_im,x4_im,numericFeatures,scaledNumericFeatures,x3_scaled,x4_scaled,is_x1_im_a,is_x1_im_c,is_x1_im_b,is_x2_im_A,is_x2_im_B
id,,,,,,,,,,,,,,,,,
1,a,A,56.0,175.0,a,A,56.0,175.0,"[56.0, 175.0]","[-0.373648934924, 0.0468807230939]",-0.373649,0.046881,1.0,0.0,0.0,1.0,0.0
2,a,B,66.0,NaN,a,B,66.0,173.0,"[66.0, 173.0]","[0.725318520735, -0.421926507845]",0.725319,-0.421927,1.0,0.0,0.0,0.0,1.0
3,b,B,NaN,182.0,b,B,56.0,182.0,"[56.0, 182.0]","[-0.373648934924, 1.68770603138]",-0.373649,1.687706,0.0,0.0,1.0,0.0,1.0
4,c,None,71.0,171.0,c,B,71.0,171.0,"[71.0, 171.0]","[1.27480224857, -0.890733738783]",1.274802,-0.890734,0.0,1.0,0.0,0.0,1.0
5,None,B,48.0,173.0,a,B,48.0,173.0,"[48.0, 173.0]","[-1.25282289945, -0.421926507845]",-1.252823,-0.421927,1.0,0.0,0.0,0.0,1.0


In [12]:
final_features = ['x3_scaled', 'x4_scaled', 'is_x1_im_a', 'is_x1_im_c', 'is_x1_im_b', 'is_x2_im_A', 'is_x2_im_B']

In [13]:
df_with_features_pd[final_features]

,x3_scaled,x4_scaled,is_x1_im_a,is_x1_im_c,is_x1_im_b,is_x2_im_A,is_x2_im_B
id,,,,,,,
1,-0.373649,0.046881,1.0,0.0,0.0,1.0,0.0
2,0.725319,-0.421927,1.0,0.0,0.0,0.0,1.0
3,-0.373649,1.687706,0.0,0.0,1.0,0.0,1.0
4,1.274802,-0.890734,0.0,1.0,0.0,0.0,1.0
5,-1.252823,-0.421927,1.0,0.0,0.0,0.0,1.0


# Pipeline IO

In [30]:
pipelineModel \
    .select(*(['id'] + final_features)) \
    .write().overwrite().save('tmp/final')

AttributeError: 'PipelineModel' object has no attribute 'select'

In [29]:
PipelineModel.load('tmp/final').transform(df).toPandas().set_index('id')[final_features]

,x3_scaled,x4_scaled,is_x1_im_a,is_x1_im_c,is_x1_im_b,is_x2_im_A,is_x2_im_B
id,,,,,,,
1,-0.373649,0.046881,1.0,0.0,0.0,1.0,0.0
2,0.725319,-0.421927,1.0,0.0,0.0,0.0,1.0
3,-0.373649,1.687706,0.0,0.0,1.0,0.0,1.0
4,1.274802,-0.890734,0.0,1.0,0.0,0.0,1.0
5,-1.252823,-0.421927,1.0,0.0,0.0,0.0,1.0
